In [32]:
import pandas as pd
from pulp import *
import random

Read the USDA dataset

In [50]:
df = pd.read_csv("USDA.csv")

PrePocessing data with delete empty or duplicate data

In [34]:
df = df.dropna(axis=0, how='any')
df.drop_duplicates(subset=None, inplace=True)

Create a dictionary to store activity multipliers based on activity level

In [ ]:
activity_multipliers = {
    'sedentary': 1.2,
    'lightly_active': 1.375,
    'moderately_active': 1.55,
    'very_active': 1.725,
    'extra_active': 1.9
}

Take input from the user

In [38]:
age = int(input("Enter your age: "))
gender = input("Enter your gender (M/F): ")
height = float(input("Enter your height (in cm): "))
weight = float(input("Enter your weight (in kg): "))
activity_level = input("Enter your activity level (sedentary, lightly_active, moderately_active, very_active, extra_active): ")

Enter your age: 23
Enter your gender (M/F): m
Enter your height (in cm): 188
Enter your weight (in kg): 95
Enter your activity level (sedentary, lightly_active, moderately_active, very_active, extra_active): very_active


I implemented the requested constraint using the "Dietary Guidelines for Americans, 2020-2025. 9th edition" guideline.
Based on the concepts outlined in the guideline and the research I have conducted, we will utilize the concept of Dietary Reference Intakes (DRI).

Now,
1.What is DRI ?

DRI stands for the dietary reference intake, sometimes incorrectly called the daily recommended intake. National Academy introduced the DRIs in 1997 - they serve as a source of multiple daily nutritional requirements in both the US and Canada. To put it simply, the DRIs tell us how much water, calories, macronutrients, and vitamins a given person should consume daily - the entire estimation is based on sex, age, and activity level.

2.How to calculate DRI?

In order to calculate your daily nutrition, we need to know your daily caloric needs.

The amount of calories you should consume daily, multiplied by your activity level allows us to:

Calculate your protein intake
10-30% of total calories (for adults)

Amount of fats
20-35% of total calories (for adults)

Consumption of carbohydrates
45-65% of total calories

Amount of iron
0.3-0.6% of total calories

3.What is BMR ? 

BMR stands for Basal Metabolic Rate. It represents the amount of energy (calories) that your body needs to maintain basic physiological functions while at rest. These functions include breathing, circulating blood, regulating body temperature, and supporting organ functions.

4.What is TTE ?

TTE stands for Total Energy Expenditure. It represents the total amount of energy (calories) that your body needs in a day, taking into account your Basal Metabolic Rate (BMR) and your activity level.

TTE is calculated by multiplying your BMR by an activity level multiplier. The activity level multiplier adjusts your BMR based on your physical activity level throughout the day. The activity levels commonly used include:

Sedentary: Little to no exercise or physical activity.
Lightly Active: Light exercise or sports 1-3 days per week.
Moderately Active: Moderate exercise or sports 3-5 days per week.
Very Active: Hard exercise or sports 6-7 days per week.
Extra Active: Very hard exercise or a physically demanding job or lifestyle.

By multiplying your BMR by the activity level multiplier corresponding to your activity level, TTE provides an estimate of the total number of calories you need to maintain your current weight and activity level. It helps guide you in managing your calorie intake and designing an appropriate meal plan or dietary regimen.

Now,Calculate BMR based on age, gender, height, and weight :

In [39]:
if gender.lower() == 'm':
    bmr = 88.362 + (13.397 * weight) + (4.799 * height) - (5.677 * age)
else:
    bmr = 447.593 + (9.247 * weight) + (3.098 * height) - (4.330 * age)

Calculate TTE by multiplying BMR with activity level multiplier

In [40]:
activity_multiplier = activity_multipliers.get(activity_level.lower(), 1.2)
tte = bmr * activity_multiplier

print("Basal Metabolic Rate (BMR):", bmr)
print("Total Energy Expenditure (TTE):", tte)

Basal Metabolic Rate (BMR): 2132.7180000000003
Total Energy Expenditure (TTE): 3678.9385500000008


Create the PuLP problem , we need to use LpMinimize.

In [41]:
prob = LpProblem("Meal_Diet_Problem", LpMinimize)

We define the required dictionaries

In [42]:
food_items = list(df['Description'])
calories = dict(zip(food_items, df['Calories']))
protein = dict(zip(food_items, df['Protein']))
totalFat = dict(zip(food_items, df['TotalFat']))
carbohydrate = dict(zip(food_items, df['Carbohydrate']))
iron = dict(zip(food_items, df['Iron']))
food_vars = LpVariable.dicts("Food", food_items, 0, cat='Integer')

Set constraints for Fat, Carbohydrate, Protein, and Iron based on DRI

In [43]:
fat_dri = 0.25 * tte / 9  # DRI for Fat (25% of TTE)
carb_dri = 0.5 * tte / 4  # DRI for Carbohydrate (50% of TTE)
protein_dri = 0.15 * tte / 4  # DRI for Protein (15% of TTE)
iron_dri = 0.0043 * tte  # DRI for Iron (0.43% of TTE)

# Define maximum constraint values
max_fat = 0.35 * tte / 9
max_carb = 0.65 * tte / 4
max_protein = 0.3 * tte / 4
max_iron = 2 * (0.0043 * tte)

Set the objective function to minimize total calories

In [44]:
prob += lpSum([calories[f] * food_vars[f] for f in food_items]), "TotalCalories"

Adding the Calorie,Fat,Carbohydrate,Protein and Iron constraints to the problem

In [45]:
# Calories
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) >= tte-200, "CaloriesMinimum"
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) <= tte + 200, "CaloriesMaximum"

# Fat
prob += lpSum([totalFat[f] * food_vars[f] for f in food_items]) >= fat_dri, "FatMinimum"
prob += lpSum([totalFat[f] * food_vars[f] for f in food_items]) <= max_fat, "FatMaximum"

# Carbohydrate
prob += lpSum([carbohydrate[f] * food_vars[f] for f in food_items]) >= carb_dri, "CarbsMinimum"
prob += lpSum([carbohydrate[f] * food_vars[f] for f in food_items]) <= max_carb, "CarbsMaximum"

# Protein
prob += lpSum([protein[f] * food_vars[f] for f in food_items]) >= protein_dri, "ProteinMinimum"
prob += lpSum([protein[f] * food_vars[f] for f in food_items]) <= max_protein, "ProteinMaximum"

# Iron
prob += lpSum([iron[f] * food_vars[f] for f in food_items]) >= iron_dri, "IronMinimum"
prob += lpSum([iron[f] * food_vars[f] for f in food_items]) <= max_iron, "IronMaximum"

The problem is solved using PuLP's Solver

In [46]:
prob.solve()

1

Print the problem solution status

In [47]:
print("Status:", LpStatus[prob.status])

Status: Optimal


Now that the problem has been resolved, we want to create a dietary plan.

In [51]:
meal_percentages = {
    'breakfast': 0.25,
    'morning_snack': 0.1,
    'lunch': 0.3,
    'afternoon_snack': 0.1,
    'dinner': 0.25
}


if LpStatus[prob.status] == 'Optimal':
    total_calories = value(prob.objective)
    total_fat = 0
    total_carbohydrate = 0
    total_iron = 0

    # Create empty meal lists
    meals = {meal: [] for meal in meal_percentages.keys()}

    # Distribute food items into meals
    for food_item in food_items:
        food_var = food_vars[food_item]
        if food_var.varValue is not None and food_var.varValue > 0:
            food_amount = food_var.varValue

            # Select a random meal based on the percentages
            meal = random.choices(list(meal_percentages.keys()), list(meal_percentages.values()))[0]

            # Append the food item to the selected meal
            meals[meal].append((food_item, food_amount))

            # Calculate total nutritional values
            food_fat = totalFat.get(food_item, 0)
            food_carbohydrate = carbohydrate.get(food_item, 0)
            food_iron = iron.get(food_item, 0)

            total_fat += food_amount * food_fat
            total_carbohydrate += food_amount * food_carbohydrate
            total_iron += food_amount * food_iron

    # Check if any meals are empty and assign one food item randomly to each empty meal
    empty_meals = [meal for meal, items in meals.items() if not items]
    for meal in empty_meals:
        food_item = random.choice(food_items)
        food_amount = 1  # Assign a fixed amount of 1 for simplicity

        meals[meal].append((food_item, food_amount))

        # Update the total nutritional values
        food_fat = totalFat.get(food_item, 0)
        food_carbohydrate = carbohydrate.get(food_item, 0)
        food_iron = iron.get(food_item, 0)

        total_fat += food_amount * food_fat
        total_carbohydrate += food_amount * food_carbohydrate
        total_iron += food_amount * food_iron

    print("-" * 110)
    print("Total Calories:", total_calories)
    print("Total Fat:", total_fat)
    print("Total Carbohydrate:", total_carbohydrate)
    print("Total Iron:", total_iron)

    # Print the final meal plan
    print("\nMeal Plan:")
    print("-" * 110)
    for meal, items in meals.items():
        meal_calories = sum(calories[food_item] * food_amount for food_item, food_amount in items)
        meal_fat = sum(totalFat[food_item] * food_amount for food_item, food_amount in items)
        meal_carbohydrate = sum(carbohydrate[food_item] * food_amount for food_item, food_amount in items)
        meal_iron = sum(iron[food_item] * food_amount for food_item, food_amount in items)

        print(f"{meal.capitalize()}:")
        for food_item, food_amount in items:
            print(f"- {food_item}: {food_amount}")

        print(f"Total Calories: {meal_calories}")
        print(f"Total Fat: {meal_fat} g")
        print(f"Total Carbohydrate: {meal_carbohydrate} g")
        print(f"Total Iron: {meal_iron} mg")
        print("-" * 110)
else:
    print("No solution found.")

--------------------------------------------------------------------------------------------------------------
Total Calories: 3479.0
Total Fat: 115.3
Total Carbohydrate: 528.12
Total Iron: 31.629999999999995

Meal Plan:
--------------------------------------------------------------------------------------------------------------
Breakfast:
- MARGARINE-LIKE,VEG OIL SPRD,20% FAT,W/ SALT: 1.0
- ALCOHOLIC BEV,BEER,LT: 2.0
Total Calories: 233.0
Total Fat: 19.5 g
Total Carbohydrate: 3.6799999999999997 g
Total Iron: 0.06 mg
--------------------------------------------------------------------------------------------------------------
Morning_snack:
- ICE CREAMS,VANILLA,FAT FREE: 5.0
Total Calories: 690.0
Total Fat: 0.0 g
Total Carbohydrate: 150.29999999999998 g
Total Iron: 0.0 mg
--------------------------------------------------------------------------------------------------------------
Lunch:
- MILK,FLUID,1% FAT,WO/ ADDED VIT A & VIT D: 2.0
- PEPPER,RED OR CAYENNE: 4.0
- ALCOHOLIC BEV,DIST